## [지정 경로에 모델을 배치하고 실행하는 경우]


```
# 다음 경로에 모델 파일을 배치하고 실행 요망.
('/content/drive/MyDrive/finalmodel/model_kobert')
```



### 모델 불러오기

In [ ]:
from tensorflow.keras.models import load_model

# 저장한 모델 불러오기
model_attention_lstm = load_model(save_path_attention_lstm, custom_objects={"AttentionLayer": AttentionLayer})

# 모델 요약 출력
model_attention_lstm.summary()

print(f"모델이 '{save_path_attention_lstm}' 경로에서 불러와졌습니다.")

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification

# 저장한 모델과 토크나이저 불러오기
loaded_tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/finalmodel/model_kobert')
loaded_model = TFBertForSequenceClassification.from_pretrained('/content/drive/MyDrive/finalmodel/model_kobert')

print("모델과 토크나이저가 불러와졌습니다.")

### 모델 앙상블

In [ ]:
from sklearn.metrics import accuracy_score, log_loss

# 모델 컴파일
loaded_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 테스트 데이터의 인코딩
test_encodings = loaded_tokenizer(list(X_test), truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors="tf")

# 각 모델의 예측 결과 계산
prob_kobert = loaded_model.predict([test_encodings['input_ids'], test_encodings['attention_mask']]).logits
prob_attention_lstm = model_attention_lstm.predict(X_test_padded)

# 로짓 값을 확률로 변환
probabilities_kobert = tf.nn.sigmoid(prob_kobert)
probabilities_attention_lstm = tf.nn.sigmoid(prob_attention_lstm)

# 앙상블을 위한 예측 확률 계산
ensemble_probabilities = (probabilities_kobert + probabilities_attention_lstm) / 2

# 앙상블 결과를 소프트 보팅하여 예측 클래스 계산
ensemble_predicted_classes = np.argmax(ensemble_probabilities, axis=1)

# 정확도 평가
ensemble_accuracy = accuracy_score(y_test, ensemble_predicted_classes)
print(f"모델 앙상블의 예측 정확도: {ensemble_accuracy:.4f}")

# 로스 평가
ensemble_loss = log_loss(y_test, ensemble_probabilities)
print(f"모델 앙상블의 로스: {ensemble_loss:.4f}")

### 테스트

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
import numpy as np
import tensorflow as tf

# 불러온 모델 및 토크나이저 설정
loaded_tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/finalmodel/model_kobert')
loaded_model = TFBertForSequenceClassification.from_pretrained('/content/drive/MyDrive/finalmodel/model_kobert')

# 텍스트 입력 받기
text = input("예측할 텍스트를 입력하세요: ")

# 입력 텍스트의 전처리 및 인코딩
encoded_input = loaded_tokenizer(text, truncation=True, padding=True, max_length=128, return_tensors="tf")

# 각 모델의 예측 결과 계산
prob_kobert = loaded_model.predict([encoded_input['input_ids'], encoded_input['attention_mask']]).logits
prob_attention_lstm = model_attention_lstm.predict(X_test_padded)

# 로짓 값을 확률로 변환
probabilities_kobert = tf.nn.sigmoid(prob_kobert)
probabilities_attention_lstm = tf.nn.sigmoid(prob_attention_lstm)

# 앙상블을 위한 예측 확률 계산
ensemble_probabilities = (probabilities_kobert + probabilities_attention_lstm) / 2

# 앙상블 결과를 소프트 보팅하여 예측 클래스 계산
ensemble_predicted_class = np.argmax(ensemble_probabilities, axis=1)[0]

# 예측 클래스와 확률 출력
class_names = ['정상', '스팸']  # 클래스 이름 설정
predicted_class_name = class_names[ensemble_predicted_class]
predicted_class_probability = ensemble_probabilities[0][ensemble_predicted_class]
print(f"예측: {predicted_class_name}")
print(f"확률: {predicted_class_probability:.4f}")

## [모델 없이 데이터셋만 있는 경우]
(이 경우는 데이터셋 CSV 파일은 있어야 합니다.)

###1. 학습 준비

#### 1.1. 모듈 및 데이터셋 불러오기

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# df_97이라는 데이터가 있다고 가정
df_97 = pd.read_csv("/content/drive/MyDrive/spam_data_free/combine_spam_labeled.csv")

In [7]:
df_97.shape

(494615, 2)

#### 1.2. 샘플링(샘플 데이터 학습 시에만 실행)

In [8]:
import pandas as pd

# 라벨 0과 라벨 1 데이터를 분리합니다.
df_97_label0 = df_97[df_97['label'] == 0]
df_97_label1 = df_97[df_97['label'] == 1]

# 각 라벨별로 10만 개씩 무작위로 샘플링합니다.
sampled_label0 = df_97_label0.sample(n=20000, random_state=42)
sampled_label1 = df_97_label1.sample(n=20000, random_state=42)

# 샘플링된 데이터를 합쳐서 새로운 데이터셋을 생성합니다.
df_99 = pd.concat([sampled_label0, sampled_label1], axis=0).sample(frac=1, random_state=42).reset_index(drop=True)

print(df_99['label'].value_counts())


1    20000
0    20000
Name: label, dtype: int64


In [10]:
df_99.shape

(40000, 2)

In [11]:
df_97 = df_99

#### 1.3. test/train 데이터 분리

In [12]:
# 데이터를 학습 데이터와 테스트 데이터로 나눕니다.
X_train, X_test, y_train, y_test = train_test_split(df_97['utterances'], df_97['label'], test_size=0.2, random_state=42)

#### 1.4. 토큰화


In [13]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 39.8 MB/s eta 0:00:00


In [14]:
import os
from konlpy.tag import Okt

# Okt 객체 생성
okt = Okt()

def tokenize(texts, saved_filename):
    """주어진 텍스트 리스트를 토큰화하고 진행 상황을 출력합니다."""
    tokenized_texts = []

    # 이전에 저장된 토큰화 결과가 있다면 불러옵니다.
    if os.path.exists(saved_filename):
        with open(saved_filename, 'r', encoding='utf-8') as f:
            tokenized_texts = [line.strip() for line in f]

    start_idx = len(tokenized_texts)
    for idx, text in enumerate(texts[start_idx:]):
        if (idx + 1) % 200 == 0:
            print(f"{idx + 1 + start_idx}개의 텍스트를 토큰화했습니다.")
            # 중간 결과를 저장합니다.
            with open(saved_filename, 'a', encoding='utf-8') as f:
                for tokens in tokenized_texts[idx-199:idx+1]:
                    f.write(' '.join(tokens) + '\n')

        tokenized_texts.append(okt.morphs(text))

    return tokenized_texts

# 학습 데이터와 테스트 데이터를 토큰화합니다.
X_train_tokenized = tokenize(X_train, 'X_train_tokenized.txt')
X_test_tokenized = tokenize(X_test, 'X_test_tokenized.txt')

print("토큰화 완료!")


200개의 텍스트를 토큰화했습니다.
400개의 텍스트를 토큰화했습니다.
600개의 텍스트를 토큰화했습니다.
800개의 텍스트를 토큰화했습니다.
1000개의 텍스트를 토큰화했습니다.
1200개의 텍스트를 토큰화했습니다.
1400개의 텍스트를 토큰화했습니다.
1600개의 텍스트를 토큰화했습니다.
1800개의 텍스트를 토큰화했습니다.
2000개의 텍스트를 토큰화했습니다.
2200개의 텍스트를 토큰화했습니다.
2400개의 텍스트를 토큰화했습니다.
2600개의 텍스트를 토큰화했습니다.
2800개의 텍스트를 토큰화했습니다.
3000개의 텍스트를 토큰화했습니다.
3200개의 텍스트를 토큰화했습니다.
3400개의 텍스트를 토큰화했습니다.
3600개의 텍스트를 토큰화했습니다.
3800개의 텍스트를 토큰화했습니다.
4000개의 텍스트를 토큰화했습니다.
4200개의 텍스트를 토큰화했습니다.
4400개의 텍스트를 토큰화했습니다.
4600개의 텍스트를 토큰화했습니다.
4800개의 텍스트를 토큰화했습니다.
5000개의 텍스트를 토큰화했습니다.
5200개의 텍스트를 토큰화했습니다.
5400개의 텍스트를 토큰화했습니다.
5600개의 텍스트를 토큰화했습니다.
5800개의 텍스트를 토큰화했습니다.
6000개의 텍스트를 토큰화했습니다.
6200개의 텍스트를 토큰화했습니다.
6400개의 텍스트를 토큰화했습니다.
6600개의 텍스트를 토큰화했습니다.
6800개의 텍스트를 토큰화했습니다.
7000개의 텍스트를 토큰화했습니다.
7200개의 텍스트를 토큰화했습니다.
7400개의 텍스트를 토큰화했습니다.
7600개의 텍스트를 토큰화했습니다.
7800개의 텍스트를 토큰화했습니다.
8000개의 텍스트를 토큰화했습니다.
8200개의 텍스트를 토큰화했습니다.
8400개의 텍스트를 토큰화했습니다.
8600개의 텍스트를 토큰화했습니다.
8800개의 텍스트를 토큰화했습니다.
9000개의 텍스트를 토큰화했습니다.
9200개의 텍스트를 토큰화했습니다.
9400개의 텍스트를 토큰화했습니다.
9600개의 텍스트를 토큰화했습

### 2 Attention + LSTM 모델


#### 2.1. 모델 설계


2.1.1. Attention 매커니즘

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Attention 가중치를 위한 weight 생성
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1), initializer="zeros")
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        # Attention score 계산
        e = tf.keras.backend.tanh(tf.keras.backend.dot(x, self.W) + self.b)
        a = tf.keras.backend.softmax(e, axis=1)
        output = x * a
        return tf.keras.backend.sum(output, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])


2.1.2. LSTM 모델 설계

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model

# 토큰화된 결과를 숫자 시퀀스로 변환
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_tokenized)

X_train_sequences = tokenizer.texts_to_sequences(X_train_tokenized)
X_test_sequences = tokenizer.texts_to_sequences(X_test_tokenized)

# 패딩 처리
MAX_LENGTH = max(len(s) for s in X_train_sequences)
X_train_padded = pad_sequences(X_train_sequences, maxlen=MAX_LENGTH, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=MAX_LENGTH, padding='post')

# 변수 정의
VOCAB_SIZE = len(tokenizer.word_index) + 1  # 단어의 개수 + 1 (0을 고려하여)
EMBEDDING_DIM = 128  # 임베딩 차원

# 모델 구성
input_layer = Input(shape=(MAX_LENGTH,))
embedding_layer = Embedding(VOCAB_SIZE, EMBEDDING_DIM)(input_layer)
lstm_layer = LSTM(128, return_sequences=True)(embedding_layer)
attention_layer = AttentionLayer()(lstm_layer)
output_layer = Dense(1, activation="sigmoid")(attention_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1908)]            0         
                                                                 
 embedding (Embedding)       (None, 1908, 128)         15771008  
                                                                 
 lstm (LSTM)                 (None, 1908, 128)         131584    
                                                                 
 attention_layer (AttentionL  (None, 128)              2036      
 ayer)                                                           
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 15,904,757
Trainable params: 15,904,757
Non-trainable params: 0
_________________________________________________

#### 2.2 학습하기

In [19]:
from tensorflow.keras.layers import Dropout

input_layer = Input(shape=(MAX_LENGTH,))
embedding_layer = Embedding(VOCAB_SIZE, EMBEDDING_DIM)(input_layer)
lstm_layer = LSTM(128, return_sequences=True)(embedding_layer)
attention_layer = AttentionLayer()(lstm_layer)

# Dropout 추가하여 적정 성능 유도
dropout_layer = Dropout(0.5)(attention_layer)
output_layer = Dense(1, activation="sigmoid")(dropout_layer)

model_attention_lstm = Model(inputs=input_layer, outputs=output_layer)
model_attention_lstm.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [20]:
# Early Stopping을 사용하여 성능 향상이 없을 때 학습을 조기 종료하는 기법을 적용하였습니다.
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# 모델 학습
history_attention_lstm = model_attention_lstm.fit(
    X_train_padded, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test_padded, y_test),
    callbacks=[early_stopping]
)

Epoch 1/20
1000/1000 [==============================] - 182s 174ms/step - loss: 0.0766 - accuracy: 0.9705 - val_loss: 0.0235 - val_accuracy: 0.9962
Epoch 2/20
1000/1000 [==============================] - 100s 100ms/step - loss: 0.0288 - accuracy: 0.9954 - val_loss: 0.0432 - val_accuracy: 0.9921
Epoch 3/20
1000/1000 [==============================] - 82s 82ms/step - loss: 0.0146 - accuracy: 0.9980 - val_loss: 0.0342 - val_accuracy: 0.9944
Epoch 4/20
1000/1000 [==============================] - 79s 79ms/step - loss: 0.0208 - accuracy: 0.9966 - val_loss: 0.0415 - val_accuracy: 0.9930


#### 2.3. SAVE/LOAD

2.3.1. save

In [180]:
# 모델 저장
save_path_attention_lstm = '/content/drive/MyDrive/finalmodel/attention_lstm_model.h5'
model_attention_lstm.save(save_path_attention_lstm)

print(f"모델이 '{save_path_attention_lstm}' 경로에 저장되었습니다.")

모델이 '/content/drive/MyDrive/finalmodel/attention_lstm_model.h5' 경로에 저장되었습니다.


2.3.2. load

In [181]:
from tensorflow.keras.models import load_model

# 저장한 모델 불러오기
model_attention_lstm = load_model(save_path_attention_lstm, custom_objects={"AttentionLayer": AttentionLayer})

# 모델 요약 출력
model_attention_lstm.summary()

print(f"모델이 '{save_path_attention_lstm}' 경로에서 불러와졌습니다.")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1908)]            0         
                                                                 
 embedding_1 (Embedding)     (None, 1908, 128)         15771008  
                                                                 
 lstm_1 (LSTM)               (None, 1908, 128)         131584    
                                                                 
 attention_layer_1 (Attentio  (None, 128)              2036      
 nLayer)                                                         
                                                                 
 dropout_38 (Dropout)        (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                           

#### 2.4. 성능 평가

In [148]:
loss, accuracy = model_attention_lstm.evaluate(X_test_padded, y_test, batch_size=32)

print(f"Attention + LSTM 모델의 테스트 손실: {loss:.4f}")
print(f"Attention + LSTM 모델의 테스트 정확도: {accuracy:.4f}")

250/250 [==============================] - 6s 26ms/step - loss: 0.0415 - accuracy: 0.9930
Attention + LSTM 모델의 테스트 손실: 0.0415
Attention + LSTM 모델의 테스트 정확도: 0.9930


### 3.  KoBERT 모델

#### 3.1.모델 구축

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.1 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam

# 학습률 조정
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
# 모델 구성 및 컴파일
model_kobert.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model_kobert.summary()

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  92186880  
                                                                 
 dropout_190 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 92,188,418
Trainable params: 92,188,418
Non-trainable params: 0
_________________________________________________________________


#### 3.2. 학습하기

In [22]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd

In [ ]:
# KoBERT 토크나이저와 모델 로드
tokenizer_kobert = BertTokenizer.from_pretrained('monologg/kobert')
model_kobert = TFBertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=2) # num_labels는 분류할 레이블의 수에 따라 조정하세요.

# 하이퍼파라미터 설정
MAX_LENGTH = 128  # 원하는 문장 최대 길이 설정

# KoBERT를 사용할 때, 입력 데이터는 토큰화 후 패딩 처리된 것이 아닌 원래의 텍스트 데이터를 사용해야 합니다.
train_encodings = tokenizer_kobert(list(X_train), truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors="tf")
test_encodings = tokenizer_kobert(list(X_test), truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors="tf")

# Early Stopping을 사용하여 성능 향상이 없을 때 학습을 조기 종료하는 기법을 적용하였습니다.
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']

model_kobert.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [40]:
# 모델 학습
history_kobert = model_kobert.fit(
    [train_encodings['input_ids'], train_encodings['attention_mask']], y_train,
    epochs=12,
    batch_size=32,
    validation_data=([test_encodings['input_ids'], test_encodings['attention_mask']], y_test),
    callbacks=[early_stopping]
)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/12
1000/1000 [==============================] - 2574s 855ms/step - loss: 0.0516 - accuracy: 0.9870 - val_loss: 0.1398 - val_accuracy: 0.9649
Epoch 2/12
1000/1000 [==============================] - 864s 865ms/step - loss: 0.1291 - accuracy: 0.9671 - val_loss: 0.1258 - val_accuracy: 0.9668
Epoch 3/12
1000/1000 [==============================] - 863s 863ms/step - loss: 0.1174 - accuracy: 0.9698 - val_loss: 0.1271 - val_accuracy: 0.9664
Epoch 4/12
1000/1000 [==============================] - 863s 863ms/step - loss: 0.1346 - accuracy: 0.9660 - val_loss: 0.1636 - val_accuracy: 0.9594
Epoch 5/12
1000/1000 [==============================] - 862s 862ms/step - loss: 0.1197 - accuracy: 0.9683 - val_loss: 0.1332 - val_accuracy: 0.9661


#### 3.3. SAVE/LOAD

save

In [176]:
# 모델 저장하기
save_path = ('/content/drive/MyDrive/finalmodel/model_kobert')
model_kobert.save_pretrained(save_path)
tokenizer_kobert.save_pretrained(save_path)

print(f"모델과 토크나이저가 '{save_path}' 경로에 저장되었습니다.")

모델과 토크나이저가 '/content/drive/MyDrive/finalmodel/model_kobert' 경로에 저장되었습니다.


load

In [170]:
from transformers import BertTokenizer, TFBertForSequenceClassification

# 저장한 모델과 토크나이저 불러오기
loaded_tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/finalmodel/model_kobert')
loaded_model = TFBertForSequenceClassification.from_pretrained('/content/drive/MyDrive/finalmodel/model_kobert')

print("모델과 토크나이저가 불러와졌습니다.")

Some layers from the model checkpoint at /content/drive/MyDrive/finalmodel/model_kobert were not used when initializing TFBertForSequenceClassification: ['dropout_228']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/finalmodel/model_kobert.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


모델과 토크나이저가 불러와졌습니다.


모델 초기화 (필요 시)

In [130]:
from transformers import BertTokenizer, TFBertForSequenceClassification

# 원본 모델과 토크나이저 로드
original_tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
original_model = TFBertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=2)

# 로드한 모델의 가중치를 원본 모델의 가중치로 복사
loaded_model.set_weights(original_model.get_weights())
loaded_tokenizer = original_tokenizer

print("모델과 토크나이저가 원본 모델에서 초기화되었습니다.")

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


모델과 토크나이저가 원본 모델에서 초기화되었습니다.


#### 3.4. 평가하기

In [172]:
from sklearn.metrics import accuracy_score

# 모델 컴파일
loaded_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 테스트 데이터의 인코딩
test_encodings = loaded_tokenizer(list(X_test), truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors="tf")

# 모델 성능 평가
loss, accuracy = model_kobert.evaluate(
    [test_encodings['input_ids'], test_encodings['attention_mask']], y_test
)

print(f"테스트 데이터에 대한 손실: {loss:.4f}")
print(f"테스트 데이터에 대한 정확도: {accuracy:.4f}")


250/250 [==============================] - 60s 241ms/step - loss: 0.1332 - accuracy: 0.9661
테스트 데이터에 대한 손실: 0.1332
테스트 데이터에 대한 정확도: 0.9661


### 4. 앙상블

#### 소프트 보팅 방법 채택

In [147]:
from sklearn.metrics import accuracy_score, log_loss

# 모델 컴파일
loaded_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 테스트 데이터의 인코딩
test_encodings = loaded_tokenizer(list(X_test), truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors="tf")

# 각 모델의 예측 결과 계산
prob_kobert = loaded_model.predict([test_encodings['input_ids'], test_encodings['attention_mask']]).logits
prob_attention_lstm = model_attention_lstm.predict(X_test_padded)

# 로짓 값을 확률로 변환
probabilities_kobert = tf.nn.sigmoid(prob_kobert)
probabilities_attention_lstm = tf.nn.sigmoid(prob_attention_lstm)

# 앙상블을 위한 예측 확률 계산
ensemble_probabilities = (probabilities_kobert + probabilities_attention_lstm) / 2

# 앙상블 결과를 소프트 보팅하여 예측 클래스 계산
ensemble_predicted_classes = np.argmax(ensemble_probabilities, axis=1)

# 정확도 평가
ensemble_accuracy = accuracy_score(y_test, ensemble_predicted_classes)
print(f"모델 앙상블의 예측 정확도: {ensemble_accuracy:.4f}")

# 로스 평가
ensemble_loss = log_loss(y_test, ensemble_probabilities)
print(f"모델 앙상블의 로스: {ensemble_loss:.4f}")

250/250 [==============================] - 6s 23ms/step
모델 앙상블의 예측 정확도: 0.9661
모델 앙상블의 로스: 0.4405


### 5. 최종 평가

#### 5.1. 모델 평가

In [157]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 정확도 평가
ensemble_accuracy = accuracy_score(y_test, ensemble_predicted_classes)
print(f"앙상블 모델의 예측 정확도: {ensemble_accuracy:.4f}")

# 정밀도 평가
precision = precision_score(y_test, ensemble_predicted_classes)
print(f"앙상블 모델의 정밀도: {precision:.4f}")

# 재현율 평가
recall = recall_score(y_test, ensemble_predicted_classes)
print(f"앙상블 모델의 재현율: {recall:.4f}")

# F1 점수 평가
f1 = f1_score(y_test, ensemble_predicted_classes)
print(f"앙상블 모델의 F1 점수: {f1:.4f}")

앙상블 모델의 예측 정확도: 0.9661
앙상블 모델의 정밀도: 0.9992
앙상블 모델의 재현율: 0.9331
앙상블 모델의 F1 점수: 0.9650


#### 5.2. I/O **테스트**

In [175]:
from transformers import BertTokenizer, TFBertForSequenceClassification
import numpy as np
import tensorflow as tf

# 불러온 모델 및 토크나이저 설정
loaded_tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/finalmodel/model_kobert')
loaded_model = TFBertForSequenceClassification.from_pretrained('/content/drive/MyDrive/finalmodel/model_kobert')

# 텍스트 입력 받기
text = input("예측할 텍스트를 입력하세요: ")

# 입력 텍스트의 전처리 및 인코딩
encoded_input = loaded_tokenizer(text, truncation=True, padding=True, max_length=128, return_tensors="tf")

# 각 모델의 예측 결과 계산
prob_kobert = loaded_model.predict([encoded_input['input_ids'], encoded_input['attention_mask']]).logits
prob_attention_lstm = model_attention_lstm.predict(X_test_padded)

# 로짓 값을 확률로 변환
probabilities_kobert = tf.nn.sigmoid(prob_kobert)
probabilities_attention_lstm = tf.nn.sigmoid(prob_attention_lstm)

# 앙상블을 위한 예측 확률 계산
ensemble_probabilities = (probabilities_kobert + probabilities_attention_lstm) / 2

# 앙상블 결과를 소프트 보팅하여 예측 클래스 계산
ensemble_predicted_class = np.argmax(ensemble_probabilities, axis=1)[0]

# 예측 클래스와 확률 출력
class_names = ['정상', '스팸']  # 클래스 이름 설정
predicted_class_name = class_names[ensemble_predicted_class]
predicted_class_probability = ensemble_probabilities[0][ensemble_predicted_class]
print(f"예측: {predicted_class_name}")
print(f"확률: {predicted_class_probability:.4f}")

Some layers from the model checkpoint at /content/drive/MyDrive/finalmodel/model_kobert were not used when initializing TFBertForSequenceClassification: ['dropout_228']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/finalmodel/model_kobert.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


예측할 텍스트를 입력하세요: 앙상블 결과를 소프트 보팅하여 예측 클래스 계산
250/250 [==============================] - 6s 23ms/step
예측: 정상
확률: 0.6318
